За последнее домашнее задание можно набрать 50 баллов – *таким образом, можно набрать до 5 дополнительных баллов*, которые могут помочь добрать где-либо ранее потерянные баллы, если есть такая необходимость.

**Дедлайн – 24 декабря, 23:59**. *Крайне приветствуется* сдача в ранний срок – работа будет проверена практически сразу (в рабочие часы) и студенту будет выставлены итоговые баллы**


---



**1.** (5 баллов) Вы разработали модель, которая страдает низким смещением и высокой дисперсией. Как Вы думаете, какой метод/алгоритм можно в таком случае применить и почему? 

Низкий ``bias`` и высокий ``variance`` свидетельствуют о переобучении модели. Чтобы решить эту проблему можно использовать/поменять регуляризацию (чтобы ограничить обучаемые параметры), упростить модель (уменьшить количество обучаемых параметров модели) или расширить датасет.

**2.** (5 баллов) У Вас есть датафрейм `df_baguettes`, содержащий список цен на французские багеты. Но оказалось, что в этом датафрейме отсутствуют многие значения в столбце цен. По крайней мере несколько багетов имеют цену в столбце.
Напишите функцию `median_baguettes`, которая вычисляет медианную цену выбранных французских багетов вместо отсутствующих значений. 

In [48]:
import pandas as pd
import math
import numpy as np

In [28]:
import pandas as pd
def median_baguettes(df:pd.DataFrame, column:str = None) -> float:
    """
    Compute the median value for pandas dataframe column.

    ### Inputs: 
    - df: The pandas dataframe;
    - column: string name of pandas dataframe column.

    ### Return:
    -  median: float value of median for column of pandas dataframe.
    """
    values = [value for value in df[column].values if not math.isnan(value)]
    values.sort()
    n = len(values)
    if n % 2:
        median = values[n // 2]
    else: 
        median = (values[n // 2] + values[n // 2 - 1]) / 2
    return median

In [39]:
data = {"price": [10, None, None, 15, None, None, None, None, None]}

df = pd.DataFrame(data)
df.sample(3)

,price
0,10.0
6,NaN
2,NaN


In [47]:
df.price.median(), median_baguettes(df, 'price')

(12.5, 12.5)

In [42]:
df1 = df.copy()
df.fillna(median_baguettes(df, "price"), inplace=True)
df1.price.fillna(df1.price.median(), inplace=True)

In [50]:
assert np.array_equal(
    df.price.values,
    df1.price.values
), 'Values filled by handcrafted and pandas median implementations are different!'

**3.** (5 баллов) Предположим, что у Вас есть набор данных как с непрерывными, так и с категориальными переменными. Какие методы кластеризации не помогут достичь высокого качества построенной модели при описанной ситуации с данными и почему? И какие методы кластеризации Вы бы использовали? 

Классические методы кластеризации, использующие, например, Евклидово расстояние, здесь абсолютно точно бесполезны. Возможно применение нетипичных расстояний, например, обобщение Махалонобиса или Gower расстояние. Алгоритмы ``KMeans``, как и ``GMM`` (расщепление смеси распределений) в этом случае всё равно не подойдут, потому что используют среднее арифметическое (в случае ``KMeans``) и статистики (мат ожидание, дисперсию ``GMM``), что даже для приведенных выше метрик лишено смысла.
Можно использовать метод аггломеративной иерархической кластеризации с приведенными метриками, ``DBSCAN`` или ``KPrototypes``. Если не прибегать к новым методам и метрикам, то можно обойтись OneHotEncoding и PCA, после чего воспользоваться классическими методами. 

**4.** Предположим, что перед Вами стоит задача: построить модель машинного обучения для прогнозирования заполняемости отеля в любую дату.


*   (2.5 баллов) Какие данные Вы бы использовали для обучения вашей модели? 
*   (10 баллов со скелетом кода, 5 баллов с письменным описанием) Какую бы модель Вы разработали? 
*   (2.5 баллов) Как бы Вы оценивали качество модели? 


1) Потенциальные данные: 
* Сезонность
* День недели
* Расположение (удаленность от центра)
* Средняя стоимость номера
* Число комнат
* Вместимость комнат (общая)
* Вместимость комнат (средняя)
* Рейтинг отеля
* Крупные мероприятия (например, бинарный признак, есть ли мероприятия с посещаемостью > 100K человек)
* Предлагаемые услуги (ресторан, парковка, бассейн)

2) Можно подходить как со стороны временных рядов (тогда можно выбрать модели, например, ARIMA, LSTM (любая другая реккурентная сеть)), так и с помощью классических моделей, вплоть до линейной регресии:

```python
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score, GridSearchCV

pipe = make_pipeline(OneHotEncoder(), StandardScaler(), Ridge())

parameters = [ {'ridge__alpha': np.arange(0, 0.2, 0.01) } ]

from sklearn.metrics import make_scorer
scoring_func = make_scorer(mean_squared_error)
grid_search = GridSearchCV(estimator=ridge_pipe, 
                           param_grid=parameters,
                           scoring=scoring_func,
                           cv=10)
grid_search = grid_search.fit(X_train, y_train)
``` 

Оценка: ``MSE``, ``MAE``

**5.** (10 баллов) У Вас есть строка, напоминающая объявление списка словарей. Не используя Pandas, напишите функцию `read_split_from_str`, которая:

*   Читает данные и кодирует их как список словарей (данные можно сгенерировать случайно при помощи numpy, например);
*   Разделяет данные на два списка: train и test в соотношении 70:30 и, соответственно, возвращает список `[training_set,testing_set]`. 

Пример входных данных:

`list_of_dict_string = "[{'x': 0, 'y': 4}, {'x': 20, 'y': 104}, {'x': 128, 'y': 212}]"`

Пример выходных данных:

```
def read_split_from_str(list_of_dict_string) 
    [
    [{'x': 0, 'y': 4}, {'x': 20, 'y': 104}],
    [{'x': 128, 'y': 212}]
    ]
```



In [54]:
import ast
list_of_dict_string = "[{'x': 0, 'y': 4}, {'x': 20, 'y': 104}, {'x': 128, 'y': 212}]"

def read_split_from_str(list_of_dict_string: str, p: float=0.7) -> list[list[dict]]:
    
    data = ast.literal_eval(list_of_dict_string)
    border = int(p * len(data))
    return [data[:border], data[border:]]

In [55]:
read_split_from_str(list_of_dict_string=list_of_dict_string)

[[{'x': 0, 'y': 4}, {'x': 20, 'y': 104}], [{'x': 128, 'y': 212}]]

**6.** (10 баллов) Разработайте модель классификации k-ближайших соседей не используя scikit-learn, соблюдая некоторые условия: 


*   В качестве метрики близости использовано евклидово расстояние;
*   Модель обратаывает датафреймы произвольного количества строк/столбцов.

In [1]:
import numpy.typing as npt

class KNeighborsClassifier:
    def __init__(self, k=5) -> None:
        """
        ### Inputs:
        - k: The number of nearest neighbors that vote for the predicted labels.
        """
        self.k = k

    def fit(self, X_train: npt.ArrayLike, y_train: npt.ArrayLike) -> None:
        """
        Train the classifier. For k-nearest neighbors this is merely
        memorizing the training data.

        ### Inputs:
        - X: A numpy array of shape (num_train, D) containing the training data
          consisting of num_train samples each of dimension D.
        - y: A numpy array of shape (N,) containing the training labels, where
             y[i] is the label for X[i].
        """
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X: npt.ArrayLike, mode:str="matrix") -> npt.ArrayLike:
        """
        Predict labels for test data using this classifier.
        
        ### Inputs:
        - X: A numpy array of shape (num_test, D) containing test data consisting
             of num_test samples each of dimension D.
        - mode: Determines which implementation to use to compute distances
          between training points and testing points.
        ### Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        if mode == "matrix":
            dists = self.__l2_matrix(X=X)
        elif mode == "explicit":
            dists = self.__l2_distance(X=X)
        else:
            ValueError(f"Invalid value {mode} for mode")
        return self.__predict_labels(dists=dists)


    def __l2_matrix(self, X: npt.ArrayLike) -> npt.ArrayLike:
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using matrix multiplication.
        """
        return np.sqrt(-2 * X @ self.X_train.T + 
            np.sum(X ** 2, axis=1, keepdims=True) +
            np.sum(self.X_train ** 2, axis=1, keepdims=True).T)


    def __l2_distance(self, X: npt.ArrayLike) -> npt.ArrayLike:
        """
        Compute the distance between each test point in X and each training point
        in self.X_train using an explicit formula of the euclidian metric.
        Input / Output: Same as compute_distances_two_loops
        """
        num_test = X.shape[0]
        num_train = self.X_train.shape[0]
        dists = np.zeros((num_test, num_train))
        for i in range(num_test):
            dists[i, :] = np.sqrt(np.sum((X[i] - self.X_train)**2, axis=1))
        return dists

    def __predict_labels(self, dists: npt.ArrayLike) -> npt.ArrayLike:
        """
        Given a matrix of distances between test points and training points,
        predict a label for each test point.

        ### Inputs:
        - dists: A numpy array of shape (num_test, num_train) where dists[i, j]
          gives the distance betwen the ith test point and the jth training point.
        ### Returns:
        - y: A numpy array of shape (num_test,) containing predicted labels for the
          test data, where y[i] is the predicted label for the test point X[i].
        """
        num_test = dists.shape[0]
        y_pred = np.zeros(num_test)
        for i in range(num_test):
            closest_y = self.y_train[np.argsort(dists[i])[:self.k]]
            y_pred[i] = np.bincount(closest_y).argmax()
        return y_pred

Проверка

In [2]:
from sklearn import datasets
dataset = datasets.load_digits()
test_border = 100
X_train, y_train = dataset.data[test_border:], dataset.target[test_border:]
X_test, y_test = dataset.data[:test_border], dataset.target[:test_border]

In [56]:
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
preds1 = clf.predict(X_test)
preds2 = clf.predict(X_test, mode="explicit")

In [57]:
from sklearn.metrics import accuracy_score
print("Obtained accuracy score for KNN with matrix multiplication: ", accuracy_score(preds1, y_test),
      "\nAccuracy score with explicit formula of euclidian metric", accuracy_score(preds2, y_test))

Obtained accuracy score for KNN with matrix multiplication:  0.93 
Accuracy score with explicit formula of euclidian metric 0.93


In [6]:
from sklearn.neighbors import KNeighborsClassifier as KNN_sklearn

clf = KNN_sklearn()
clf.fit(X_train, y_train)

In [10]:
assert np.array_equal(
    clf.predict(X_test),
    preds1
), 'Labels predicted by handcrafted and sklearn kNN implementations are different!'

assert np.array_equal(
    clf.predict(X_test),
    preds2
), 'Labels predicted by handcrafted and sklearn kNN implementations are different!'

print("Accuracy score for sklearn implementation", accuracy_score(clf.predict(X_test), y_test))

Accuracy score for sklearn implementation 0.93
